### Import Dependencies

In [1]:
# Pygame intended for later use
!pip install pygame

In [2]:
# Gym Imports
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete # different types of spaces

# Helpers
import numpy as np
import random
import os

# Stable baselines
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [3]:
# Pygame
import pygame as pg

screen_width = 800
screen_height = 600

pg.init()
screen = pg.display.set_mode((screen_width,screen_height))
pg.display.set_caption("Shower")

# see if we can load more than standard BMP
if not pg.image.get_extended():
    raise SystemExit("Sorry, extended image module required")

# game constants
SCREENRECT = pg.Rect(0, 0, 1280, 720)

pygame 2.1.0 (SDL 2.0.16, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
#pg.quit() # run this to close pygame window

In [4]:
def load_image(file):
    """loads an image, prepares it for play"""
    file = os.path.join(file)
    try:
        surface = pg.image.load(file)
    except pg.error:
        raise SystemExit('Could not load image "%s" %s' % (file, pg.get_error()))
    return surface.convert()

In [5]:
class Knob(pg.sprite.Sprite):
    """A visual for what way the AI is turning the temperature"""
    images = [load_image("assets/sprites/knob_none.png"), 
              load_image("assets/sprites/knob_r.png"), 
              load_image("assets/sprites/knob_l.png")]

    def __init__(self):
        pg.sprite.Sprite.__init__(self, self.containers)
        self.image = self.images[0]
        self.rect = self.image.get_rect()

    def update(self, action: int):
        self.image = self.images[action]

In [6]:
# Initialize Game Groups
all = pg.sprite.RenderUpdates()
    
Knob.containers = all

### Building an Environment
- Build an agent to give us best shower possible
- Randomly temperature
- 37 and 39 degrees range

In [7]:
class ShowerEnv(Env):
    visual = Knob()
    
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60

    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1
        self.visual.update(action)
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # create the background, tile the bgd image
        bgdtile = load_image("assets/sprites/background.png")
        background = pg.Surface(SCREENRECT.size)
        for x in range(0, SCREENRECT.width, bgdtile.get_width()):
            background.blit(bgdtile, (x, 0))
        screen.blit(background, (0, 0))
        
         # draw the scene
        all.clear(screen, background) # clear/erase the last drawn sprites
        dirty = all.draw(screen)
        pg.display.update(dirty)
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

### Test Environment

In [8]:
env = ShowerEnv()

C:\Users\gameuser\anaconda3\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [17]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-58
Episode:2 Score:-28
Episode:3 Score:-24
Episode:4 Score:-20
Episode:5 Score:-58


### Train Model

In [ ]:
log_path = os.path.join('training', 'logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=100000)

### Save Model

In [12]:
shower_path = os.path.join('training', 'saved_models', 'ShowerModel_PPO')

In [ ]:
model.save(shower_path)

In [ ]:
del(model)

In [13]:
model = PPO.load(shower_path, env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

In [16]:
episodes = 10
for episode in range(1, episodes + 1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render() # renders environment
        action, _ = model.predict(obs) # predict returns two, but we only require action
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close() # Close environment, can run outside of cell to also close environment

Episode:1 Score:52
Episode:2 Score:56
Episode:3 Score:52
Episode:4 Score:58
Episode:5 Score:60
Episode:6 Score:60
Episode:7 Score:56
Episode:8 Score:60
Episode:9 Score:60
Episode:10 Score:56


In [18]:
pg.quit()